In [4]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy
import tflearn
import tensorflow
import random
import json

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [5]:
with open("intents.json") as file:
    data = json.load(file)
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Whats up'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['cya',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Have a Good day'],
   'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'],
   'context_set': ''},
  {'tag': 'age',
   'patterns': ['how old',
    'how old is Devansh',
    'what is your age',
    'how old are you',
    'age?'],
   'responses': ['I am 19 years old!',
    '19 years young!',
    'Old enough to adopt you now'],
   'context_set': ''},
  {'tag': 'name',
   'patterns': ['what is your name',
    'what should I call you',
    'whats your name?'],
   'responses': ['You can call me Devansh, or Dev, or Sweety!',
    "I'm Devansh!",
    "I'm Devansh the LEGEND!"],
   'context_set': ''},
  {'tag': 'study',
 

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Devansh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
words = []
labels=[]
docs_x=[]
docs_y=[]
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern) #splits the words in the sentences of patterns
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
    if intent["tag"] not in labels:
        labels.append(intent["tag"])

         

In [8]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))
labels = sorted(labels)

In [9]:
training = []
output = []
out_empty = [0 for _ in range(len(labels))]

for x,doc in enumerate(docs_x):
    bag=[]
    wrds = [stemmer.stem(w) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    training.append(bag)
    output.append(output_row)

In [10]:
traing = numpy.array(training)
output = numpy.array(output)

In [11]:
tensorflow.compat.v1.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]),activation="softmax")
net = tflearn.regression(net)
model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
model.fit(training,output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 6999  | total loss: 0.00391 | time: 0.028s
| Adam | epoch: 1000 | loss: 0.00391 - acc: 1.0000 -- iter: 48/52
Training Step: 7000  | total loss: 0.00374 | time: 0.032s
| Adam | epoch: 1000 | loss: 0.00374 - acc: 1.0000 -- iter: 52/52
--
INFO:tensorflow:c:\Users\Devansh\Documents\machine learning\Chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [10]:
def bag_of_words(s,words):
    bag=[0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    for se in s_words:
        for i,w in enumerate(words):
            if w==se:
                bag[i]=(1)
    return numpy.array(bag)

In [11]:
def chat():
    print("Start Talking (Type quit to stop)")
    while True:
        inp = input("You: ")
        if(inp.lower == "quit"):
            break
        res = model.predict([bag_of_words(inp,words)])
        res_index = numpy.argmax(res)
        tag = labels[res_index]
        for tg in data["intents"]:
            if tg["tag"] == tag:
                responses = tg['responses']
        print(random.choice(responses))


In [2]:
chat()

NameError: name 'chat' is not defined

In [14]:
len(output[0])

9